Importation des library

In [1]:
import numpy as np
import pandas as pd
import vectorbt as vbt
from itertools import product
# ============================================================
# CONFIGURATION DATASETS
# ============================================================
pd.set_option('future.no_silent_downcasting', True)


DATASETS = {
    "BTCUSDT_4h": r"C:\Users\gunsa\Desktop\Git-Repo\Bot-Trading\Data\4h\BTCUSDT\BTCUSDT_1years_2024-12-22_TO_2025-12-25_4h_data",
    "ETHUSDT_4h": r"C:\Users\gunsa\Desktop\Git-Repo\Bot-Trading\Data\4h\ETHUSDT\ETHUSDT_1years_2024-12-22_TO_2025-12-26_4h_data",
    "BTCUSDT_1h": r"C:\Users\gunsa\Desktop\Git-Repo\Bot-Trading\Data\1h\BTCUSDT\BTCUSDT_1years_2024-12-22_TO_2025-12-26_1h_data",
    "ETHUSDT_1h": r"C:\Users\gunsa\Desktop\Git-Repo\Bot-Trading\Data\1h\ETHUSDT\ETHUSDT_1years_2024-12-22_TO_2025-12-26_1h_data",
}

INIT_CASH = 1000
FEES = 0.001

# === CONFIG ROBUSTE ===
RSI_LENGTH = 14
LOOKBACK_LEFT = 3
LOOKBACK_RIGHT = 3

STOP_LOSS = 0.02          # SL de sécurité
TAKE_PROFIT = None        # TP désactivé
TRAILING_STOP = 0.03      # 3% trailing

MIN_DIV_BARS = 10
MAX_DIV_BARS = 200

TRAIN_RATIO = 0.6
TEST_RATIO = 0.2

# ============================================================
# STRATÉGIE
# ============================================================

def run_strategy(df):
    high = df["High"].astype(np.float64)
    low = df["Low"].astype(np.float64)
    close = df["Close"].astype(np.float64).ffill().infer_objects(copy=False)

    rsi = vbt.RSI.run(close, window=RSI_LENGTH).rsi

    window = LOOKBACK_LEFT + LOOKBACK_RIGHT + 1

    pivot_low_price = low.shift(LOOKBACK_RIGHT) == low.rolling(window).min()
    pivot_low_rsi = rsi.shift(LOOKBACK_RIGHT) == rsi.rolling(window).min()

    bullish_div = pd.Series(False, index=df.index)
    hidden_bullish_div = pd.Series(False, index=df.index)

    idx = np.where(pivot_low_price & pivot_low_rsi)[0]

    if len(idx) >= 2:
        idx1 = idx[1:]
        idx2 = idx[:-1]
        bars_between = np.abs(idx1 - idx2)

        regular = (
            (low.iloc[idx1].values < low.iloc[idx2].values) &
            (rsi.iloc[idx1].values > rsi.iloc[idx2].values) &
            (bars_between >= MIN_DIV_BARS) &
            (bars_between <= MAX_DIV_BARS)
        )

        hidden = (
            (low.iloc[idx1].values > low.iloc[idx2].values) &
            (rsi.iloc[idx1].values < rsi.iloc[idx2].values) &
            (bars_between >= MIN_DIV_BARS) &
            (bars_between <= MAX_DIV_BARS)
        )

        bullish_div.iloc[idx1[regular] - LOOKBACK_RIGHT] = True
        hidden_bullish_div.iloc[idx1[hidden] - LOOKBACK_RIGHT] = True

    entries = (bullish_div | hidden_bullish_div)
    entries = entries.shift(1).fillna(False).astype(bool)

    pf = vbt.Portfolio.from_signals(
        close=close,
        entries=entries,
        sl_stop=STOP_LOSS,
        sl_trail=TRAILING_STOP,
        init_cash=INIT_CASH,
        fees=FEES,
        direction="longonly"
    )

    return pf

# ============================================================
# WALK FORWARD
# ============================================================

def walk_forward(df):
    n = len(df)
    train_size = int(n * TRAIN_RATIO)
    test_size = int(n * TEST_RATIO)

    results = []
    start = 0
    step = test_size
    fold = 1

    while start + train_size + test_size <= n:
        test_df = df.iloc[start + train_size:start + train_size + test_size]

        pf = run_strategy(test_df)
        stats = pf.stats()

        results.append({
            "fold": fold,
            "start": test_df.index[0],
            "end": test_df.index[-1],
            "return": stats["Total Return [%]"],
            "drawdown": stats["Max Drawdown [%]"],
            "trades": pf.trades.count()
        })

        fold += 1
        start += step

    return pd.DataFrame(results)

# ============================================================
# RUN
# ============================================================

if __name__ == "__main__":
    all_results = []

    for name, path in DATASETS.items():
        print(f"\n▶ Walk-forward avec trailing stop : {name}")
        df = pd.read_parquet(path)

        wf = walk_forward(df)
        wf["dataset"] = name
        print(wf)

        all_results.append(wf)

    final = pd.concat(all_results)

    print("\n===== RÉSUMÉ GLOBAL =====\n")
    print(
        final.groupby("dataset")[["return", "drawdown", "trades"]].mean()
    )

    final.to_parquet("walk_forward_trailing_results.parquet")


▶ Walk-forward avec trailing stop : BTCUSDT_4h
   fold                     start                       end    return  \
0     1 2025-07-31 09:00:00+01:00 2025-10-12 21:00:00+01:00 -0.623895   
1     2 2025-10-13 01:00:00+01:00 2025-12-25 13:00:00+01:00 -6.064643   

   drawdown  trades     dataset  
0  4.581064       3  BTCUSDT_4h  
1  8.401084       3  BTCUSDT_4h  

▶ Walk-forward avec trailing stop : ETHUSDT_4h
   fold                     start                       end    return  \
0     1 2025-07-31 17:00:00+01:00 2025-10-13 09:00:00+01:00  9.021854   
1     2 2025-10-13 13:00:00+01:00 2025-12-26 05:00:00+01:00 -2.502263   

   drawdown  trades     dataset  
0  2.394789       2  ETHUSDT_4h  
1  2.852695       1  ETHUSDT_4h  

▶ Walk-forward avec trailing stop : BTCUSDT_1h
   fold                     start                       end    return  \
0     1 2025-07-31 19:00:00+01:00 2025-10-13 16:00:00+01:00  4.246862   
1     2 2025-10-13 17:00:00+01:00 2025-12-26 14:00:00+01:00 -0.521